In [1]:
import numpy as np
import torch
from pytorch_tabnet.tab_model import TabNetClassifier
import pandas as pd
import kagglehub
from kagglehub import KaggleDatasetAdapter
from sklearn.model_selection import train_test_split,cross_validate
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
df = kagglehub.load_dataset(adapter=KaggleDatasetAdapter.PANDAS,handle="fedesoriano/stroke-prediction-dataset",path="healthcare-dataset-stroke-data.csv")

In [3]:
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [4]:
display(df.dtypes)
print(f"Shape: {df.shape}")


id                     int64
gender                object
age                  float64
hypertension           int64
heart_disease          int64
ever_married          object
work_type             object
Residence_type        object
avg_glucose_level    float64
bmi                  float64
smoking_status        object
stroke                 int64
dtype: object

Shape: (5110, 12)


In [5]:
df.smoking_status.unique()

array(['formerly smoked', 'never smoked', 'smokes', 'Unknown'],
      dtype=object)

In [6]:
#Binary Categorical Features to encode
categorical_binary = ['ever_married', 'Residence_type']
categorical_multi = ['work_type', 'gender','smoking_status']
numerical_double = ['age', 'avg_glucose_level', 'bmi',]

#create numerical encodings for binary categorical features
df['ever_married'] = df['ever_married'].apply(lambda x: 1 if x == 'Yes' else 0)
df['Residence_type'] = df['Residence_type'].apply(lambda x: 1 if x == 'Urban' else 0)

#one hot encode multi categorical features
df = pd.concat([df, pd.get_dummies(data= df[categorical_multi],dtype=int)], axis=1)

In [7]:
df = df.drop(columns=categorical_multi)

In [8]:
#check for missing values
df.dtypes
#convert all float64 to float32
df = df.astype({col: np.float32 for col in df.select_dtypes('float64').columns})
df.dtypes

id                                  int64
age                               float32
hypertension                        int64
heart_disease                       int64
ever_married                        int64
Residence_type                      int64
avg_glucose_level                 float32
bmi                               float32
stroke                              int64
work_type_Govt_job                  int64
work_type_Never_worked              int64
work_type_Private                   int64
work_type_Self-employed             int64
work_type_children                  int64
gender_Female                       int64
gender_Male                         int64
gender_Other                        int64
smoking_status_Unknown              int64
smoking_status_formerly smoked      int64
smoking_status_never smoked         int64
smoking_status_smokes               int64
dtype: object

In [9]:
#remove rows whith no bmi data
df = df.dropna()
X = df.drop(columns='stroke')
y = df['stroke']

#split data into train, test, and validation sets stratified by stroke
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)


In [10]:
#split data into train, test, and validation sets stratified by stroke
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)
X_train.shape

(3141, 20)

In [11]:
print(torch.backends.mps.is_available())  # Should return True


True


In [12]:
tabnet_model = TabNetClassifier(optimizer_fn=torch.optim.AdamW,
                                scheduler_params={"T_max":10, "eta_min":1e-5},  # Decay LR smoothly
                                scheduler_fn=torch.optim.lr_scheduler.CosineAnnealingLR,
                                optimizer_params=dict(lr=1e-3),n_d=32,n_a=32,n_steps=3,
                                device_name="mps",verbose=1) 

tabnet_model.fit(X_train=X_train.values, y_train=y_train.values,
                  eval_set=[(X_val.values, y_val.values)],
                  eval_name=['validation-logloss'],
                  eval_metric=['logloss'], num_workers=1, max_epochs=30,virtual_batch_size=64,
                  batch_size=256)

/opt/miniconda3/envs/mlenv/lib/python3.11/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : mps
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.67885 | validation-logloss_logloss: 0.54996 |  0:00:15s
epoch 1  | loss: 0.41611 | validation-logloss_logloss: 0.57946 |  0:00:29s
epoch 2  | loss: 0.29121 | validation-logloss_logloss: 0.4104  |  0:00:43s
epoch 3  | loss: 0.23838 | validation-logloss_logloss: 0.33161 |  0:00:58s
epoch 4  | loss: 0.21949 | validation-logloss_logloss: 0.28672 |  0:01:12s
epoch 5  | loss: 0.20081 | validation-logloss_logloss: 0.26046 |  0:01:26s
epoch 6  | loss: 0.19836 | validation-logloss_logloss: 0.24162 |  0:01:40s
epoch 7  | loss: 0.1971  | validation-logloss_logloss: 0.23575 |  0:01:54s
epoch 8  | loss: 0.18888 | validation-logloss_logloss: 0.23485 |  0:02:08s


KeyboardInterrupt: 

In [76]:
# Convert model to PyTorch format
pytorch_model = tabnet_model.network
pytorch_model.eval()

TabNet(
  (embedder): EmbeddingGenerator()
  (tabnet): TabNetNoEmbeddings(
    (initial_bn): BatchNorm1d(20, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
    (encoder): TabNetEncoder(
      (initial_bn): BatchNorm1d(20, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
      (initial_splitter): FeatTransformer(
        (shared): GLU_Block(
          (shared_layers): ModuleList(
            (0): Linear(in_features=20, out_features=64, bias=False)
            (1): Linear(in_features=32, out_features=64, bias=False)
          )
          (glu_layers): ModuleList(
            (0): GLU_Layer(
              (fc): Linear(in_features=20, out_features=64, bias=False)
              (bn): GBN(
                (bn): BatchNorm1d(64, eps=1e-05, momentum=0.02, affine=True, track_running_stats=True)
              )
            )
            (1): GLU_Layer(
              (fc): Linear(in_features=32, out_features=64, bias=False)
              (bn): GBN(
               

In [77]:
dummy_input = torch.randn(3141, 20).float()  # 3141 sample, 20 features

# Export model to ONNX
torch.onnx.export(
    pytorch_model, dummy_input, "tabnet.onnx",
    export_params=True,    # Store the trained parameters
    opset_version=11,      # Use ONNX opset 11+
    do_constant_folding=True,  # Optimize graph by folding constants
    input_names=["input"], output_names=["output"]
)

print("TabNet model has been converted to ONNX format!")

/opt/miniconda3/envs/mlenv/lib/python3.11/site-packages/pytorch_tabnet/tab_network.py:35: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  chunks = x.chunk(int(np.ceil(x.shape[0] / self.virtual_batch_size)), 0)


TabNet model has been converted to ONNX format!


In [37]:
explainability_matrix , masks = tabnet_model.explain(X_test.values)
feature_names = X_train.columns.values
fig, ax = plt.subplots(ncols=2, figsize=(100, 90))
xticks = [0,785,1570,2355,3141]


# Plot mask for step 1 (Transpose using T)
sns.heatmap(masks[0].T, ax=ax[0], cmap='viridis', cbar=False)
ax[0].set_title('Mask for Step 1', fontsize=60)
ax[0].set_ylabel('Features', fontsize=45)
ax[0].set_xlabel('Samples', fontsize=45)
ax[0].set_xticks(xticks)
ax[0].set_xticklabels(xticks, fontsize=30)
ax[0].set_yticklabels(feature_names, fontsize=40, rotation=0.45)

# Plot mask for step 2 (Transpose using T)
sns.heatmap(masks[1].T, ax=ax[1], cmap='viridis', cbar_kws={"label": "Importance"})
ax[1].set_title('Mask for Step 2', fontsize=60)
ax[1].set_ylabel('Features', fontsize=45)
ax[1].set_xlabel('Samples', fontsize=45)
ax[1].set_xticks(xticks)
ax[1].set_xticklabels(xticks, fontsize=30)
ax[1].set_yticklabels(feature_names, fontsize=40, rotation=0.45)


plt.show()


In [27]:
#Use onnx model to make predictions
import onnxruntime as ort


# Load the ONNX model
onnx_model = ort.InferenceSession("tabnet.onnx")

# Get the input name for the ONNX model
input_name = onnx_model.get_inputs()[0].name

# Get the output name for the ONNX model
output_name = onnx_model.get_outputs()[0].name

# Convert the input data to ONNX format
input_data = X_train.values

# Make predictions using ONNX model
predictions_onnx = onnx_model.run([output_name], {input_name: input_data.astype(np.float32)})[0]

#show stats
print(f"ONNX model predictions: {predictions_onnx}")

ONNX model predictions: [[ 3.3532372  -0.7668635 ]
 [ 2.6243238  -0.8884481 ]
 [ 0.9245977  -1.1461244 ]
 ...
 [ 1.3486643  -0.48778722]
 [ 0.7718145  -0.98826957]
 [ 0.88292843 -0.64041334]]
